In [1]:
import itertools
import math

from game import Cage, op, Cell
import game_parser

In [2]:
puzzle = game_parser.get_puzzle(95541)
puzzle

Puzzle(board_size=6, cages=[Cage(cells=[Cell(row=0, col=0), Cell(row=0, col=1), Cell(row=0, col=2)], op=<op.PROD: '*'>, val=90), Cage(cells=[Cell(row=0, col=3), Cell(row=1, col=3)], op=<op.SUM: '+'>, val=5), Cage(cells=[Cell(row=0, col=4), Cell(row=0, col=5)], op=<op.SUM: '+'>, val=5), Cage(cells=[Cell(row=1, col=0), Cell(row=2, col=0)], op=<op.DIV: '/'>, val=2), Cage(cells=[Cell(row=1, col=1), Cell(row=1, col=2)], op=<op.SUM: '+'>, val=11), Cage(cells=[Cell(row=1, col=4), Cell(row=1, col=5)], op=<op.DIV: '/'>, val=2), Cage(cells=[Cell(row=2, col=1), Cell(row=3, col=1)], op=<op.SUB: '-'>, val=5), Cage(cells=[Cell(row=2, col=2), Cell(row=2, col=3), Cell(row=2, col=4)], op=<op.PROD: '*'>, val=72), Cage(cells=[Cell(row=2, col=5), Cell(row=3, col=5), Cell(row=3, col=4)], op=<op.SUM: '+'>, val=10), Cage(cells=[Cell(row=3, col=0), Cell(row=4, col=0)], op=<op.PROD: '*'>, val=20), Cage(cells=[Cell(row=3, col=2), Cell(row=3, col=3)], op=<op.SUM: '+'>, val=6), Cage(cells=[Cell(row=4, col=1), Cel

In [3]:
# Validate that we have the right number of total cells, and unique cells
puzzle.board_size**2, sum(len(c.cells) for c in puzzle.cages), len(set.union(*(set(c.cells) for c in puzzle.cages)))

(36, 36, 36)

In [4]:
# convenience func
BOARD_SIZE = puzzle.board_size

def get_nums():
    return range(1, BOARD_SIZE+1)

# create a grid of possible values
possible_values = {Cell(row, col): set(get_nums()) for row in range(BOARD_SIZE) for col in range(BOARD_SIZE)}
possible_values


{Cell(row=0, col=0): {1, 2, 3, 4, 5, 6},
 Cell(row=0, col=1): {1, 2, 3, 4, 5, 6},
 Cell(row=0, col=2): {1, 2, 3, 4, 5, 6},
 Cell(row=0, col=3): {1, 2, 3, 4, 5, 6},
 Cell(row=0, col=4): {1, 2, 3, 4, 5, 6},
 Cell(row=0, col=5): {1, 2, 3, 4, 5, 6},
 Cell(row=1, col=0): {1, 2, 3, 4, 5, 6},
 Cell(row=1, col=1): {1, 2, 3, 4, 5, 6},
 Cell(row=1, col=2): {1, 2, 3, 4, 5, 6},
 Cell(row=1, col=3): {1, 2, 3, 4, 5, 6},
 Cell(row=1, col=4): {1, 2, 3, 4, 5, 6},
 Cell(row=1, col=5): {1, 2, 3, 4, 5, 6},
 Cell(row=2, col=0): {1, 2, 3, 4, 5, 6},
 Cell(row=2, col=1): {1, 2, 3, 4, 5, 6},
 Cell(row=2, col=2): {1, 2, 3, 4, 5, 6},
 Cell(row=2, col=3): {1, 2, 3, 4, 5, 6},
 Cell(row=2, col=4): {1, 2, 3, 4, 5, 6},
 Cell(row=2, col=5): {1, 2, 3, 4, 5, 6},
 Cell(row=3, col=0): {1, 2, 3, 4, 5, 6},
 Cell(row=3, col=1): {1, 2, 3, 4, 5, 6},
 Cell(row=3, col=2): {1, 2, 3, 4, 5, 6},
 Cell(row=3, col=3): {1, 2, 3, 4, 5, 6},
 Cell(row=3, col=4): {1, 2, 3, 4, 5, 6},
 Cell(row=3, col=5): {1, 2, 3, 4, 5, 6},
 Cell(row=4, col

In [5]:
# Pretty-printing the current state
def print_cell(nums):
    return "".join([str(i) if i in nums else " " for i in get_nums()])

def print_grid(values):
    for row in range(BOARD_SIZE):
        print ("|".join([print_cell(values[(row, col)]) for col in range(BOARD_SIZE)]))

print_grid(possible_values)


123456|123456|123456|123456|123456|123456
123456|123456|123456|123456|123456|123456
123456|123456|123456|123456|123456|123456
123456|123456|123456|123456|123456|123456
123456|123456|123456|123456|123456|123456
123456|123456|123456|123456|123456|123456


In [6]:
# For a cage, enumerate all the sets of numbers that could meet the arithmetic constraint
def enum_cage_solutions(cage):
    solutions = {}
    # Pass the operator, size and value
    # Use pattern matching to validate the right number of cells for an operator
    match (cage.op, len(cage.cells), cage.val):
        case (op.NULL, 1, val):
            solutions = [[val]]
        case (op.DIV, 2, val):
            solutions = [(i, i*val) for i in range(1, int(BOARD_SIZE/val)+1)]
        case (op.SUB, 2, val):
            solutions = [(i, i+val) for i in range(1, BOARD_SIZE-val+1)]
        case (op.PROD, size, val):
            factors = (i for i in get_nums() if val%i == 0)
            combos = itertools.product(factors, repeat=size)
            possibles = [tuple(sorted(combo)) for combo in combos if math.prod(combo)==val]
            solutions = list(set(possibles))
        case (op.SUM, size, val):
            possible_nums = get_nums()
            combos = itertools.product(possible_nums, repeat=size)
            possibles = [tuple(sorted(combo)) for combo in combos if sum(combo)==val]
            solutions = list(set(possibles))
        case _:
            raise RuntimeError(f"Unrecognized operator, or wrong size for operator in {cage}")
    return solutions

In [7]:
[enum_cage_solutions(s) for s in puzzle.cages]

[[(3, 5, 6)],
 [(2, 3), (1, 4)],
 [(2, 3), (1, 4)],
 [(1, 2), (2, 4), (3, 6)],
 [(5, 6)],
 [(1, 2), (2, 4), (3, 6)],
 [(1, 6)],
 [(2, 6, 6), (3, 4, 6)],
 [(2, 2, 6), (3, 3, 4), (2, 3, 5), (2, 4, 4), (1, 3, 6), (1, 4, 5)],
 [(4, 5)],
 [(2, 4), (3, 3), (1, 5)],
 [(2, 2, 4), (1, 4, 4)],
 [(1, 6)],
 [(1, 2), (2, 4), (3, 6)],
 [(1, 6), (2, 5), (3, 4)],
 [(2, 4), (3, 3), (1, 5)]]

In [8]:
# Apply the cage math
# For each cage, compute the possible values that can fill the cage,
# and restrict the global possible values to those values
def do_cage_math_logic():
    for cage in puzzle.cages:
        sols = enum_cage_solutions(cage)
        nums = set.union(*[set(sol) for sol in sols])
        for cell in cage.cells:
            possible_values[cell] = possible_values[cell].intersection(nums)

do_cage_math_logic()
print_grid(possible_values)

  3 56|  3 56|  3 56|1234  |1234  |1234  
1234 6|    56|    56|1234  |1234 6|1234 6
1234 6|1    6| 234 6| 234 6| 234 6|123456
   45 |1    6|12345 |12345 |123456|123456
   45 |12 4  |12 4  |1    6|1    6|1234 6
123456|123456|12 4  |12345 |12345 |1234 6


In [9]:
# This does the pigeonhole logic of one number per row/column
# The logic is easy when a cell has only one possible value
# But it can be generalized; if a row has two pairs with two values, those two values can't be elsewhere
# Eg the ' 23' in two cells of row 0 means that there can't be a 2 or 3 elsewhere in row 0
# And so on for 3 sets of size 3, etc
# TODO generalize this to columns
def do_pigeonhole_logic():
    for row in range(BOARD_SIZE):
        for size in range (1,4):
            pairs = {col: possible_values[(row, col)] for col in range(BOARD_SIZE) if len(possible_values[(row, col)])==size}
            paired_pairs = [pair for pair in pairs.values() if list(pairs.values()).count(pair)==size]
            nums = []
            if paired_pairs:
                nums = set.union(*[set(pair) for pair in paired_pairs])
                for col in range(BOARD_SIZE):
                    possibles = possible_values[(row, col)]
                    if possibles not in paired_pairs:
                        possible_values[(row, col)] = possibles - nums

    # For now copy/paste for cols
    for col in range(BOARD_SIZE):
        for size in range (1,4):
            pairs = {row: possible_values[(row, col)] for row in range(BOARD_SIZE) if len(possible_values[(row, col)])==size}
            paired_pairs = [pair for pair in pairs.values() if list(pairs.values()).count(pair)==size]
            nums = []
            if paired_pairs:
                nums = set.union(*[set(pair) for pair in paired_pairs])
                for row in range(BOARD_SIZE):
                    possibles = possible_values[(row, col)]
                    if possibles not in paired_pairs:
                        possible_values[(row, col)] = possibles - nums

do_pigeonhole_logic()
print_grid(possible_values)

  3  6|  3 5 |  3 56|12 4  |12 4  |12 4  
123   |    5 |    56|1234  |1234  |1234  
123  6|1    6| 234 6| 234 6| 234 6|123456
   45 |1    6|12345 |12345 |123456|123456
   45 | 2 4  | 2 4  |1    6|1    6| 234  
123  6| 2345 |12 4  |12345 |12345 |1234 6


In [10]:
# another iteration
do_pigeonhole_logic()
print_grid(possible_values)

  3  6|  3   |  3 5 |12 4  |12 4  |12 4  
123   |    5 |     6|1234  |1234  |12 4  
123  6|1    6| 234  | 234 6| 234 6|12 456
   4  |1    6|12345 |12345 |123456|12 456
    5 | 2 4  | 2 4  |1    6|1    6|  3   
123  6| 234  |12 4  |12345 |12345 |12 4 6


In [11]:
# a few more iterations
[do_pigeonhole_logic() for _ in range(5)]
print_grid(possible_values)

     6|  3   |    5 |12 4  |12 4  |12 4  
123   |    5 |     6|1234  |1234  |12 4  
123   |1    6| 234  | 234 6| 234 6|12 456
   4  |1    6|123   |123 5 |123 56|12  56
    5 | 2 4  | 2 4  |1    6|1    6|  3   
123   | 2 4  |12 4  |12345 |12345 |12 4 6


In [12]:
# TODO the cage math logic is pretty basic now
# I could expand it to see which sets are actually possible based on current state
# E.g., if a valid set is {1, 2 ,3}, but we know that 3 can't be in any of the three cells,
# we can eliminate that set

# TODO iteratively run the pigeonhole and math logic until we solve the problem (or converge without a solution)